In [1]:
import pandas as pd
import numpy as np

In [2]:
# df  = pd.read_csv('../data/hawkes_analysis_data.csv')
df  = pd.read_csv(r'I:/Hawkes_Model_MM/hawkes_analysis_data.csv')

In [3]:
df.head()

,trade_number,trade_price,volume,buy_order_number,buy_algo,buy_client,sell_order_number,sell_algo,sell_client,timestamp,buy_entry_ts,sell_entry_ts,aggressor_side,date
0,2019081325008066,785.0,1,1100000000068964,1,2,1100000000062580,1,3,2019-08-13 09:15:00.006118774,2019-08-13 09:15:00.005767822,NaN,-1,2019-08-13
1,2019081325008128,785.0,2,1100000000069441,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024169922,2019-08-13 09:15:00.024139404,NaN,-1,2019-08-13
2,2019081325008129,785.0,47,1100000000069442,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024230957,2019-08-13 09:15:00.024215698,NaN,-1,2019-08-13
3,2019081325008130,785.0,53,1100000000069442,1,3,1100000000068983,0,3,2019-08-13 09:15:00.024246216,2019-08-13 09:15:00.024215698,2019-08-13 09:15:00.005935669,1,2019-08-13
4,2019081325008466,785.0,10,1100000000055032,1,3,1100000000070554,0,1,2019-08-13 09:15:00.118347168,NaN,2019-08-13 09:15:00.118331909,1,2019-08-13


In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
dates = df['date'].unique()

In [6]:
target_date = dates[0]

In [7]:
# df_day = df[df['date'] == target_date].sort_values('timestamp')
df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])

In [8]:
df_day['timestamp'].dtype

dtype('<M8[ns]')

In [9]:
market_open = pd.to_datetime((target_date) + ' 09:15:00')
df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()

In [10]:
buy_times = df_day[df_day['aggressor_side'] == +1]['t'].to_numpy()
sell_times = df_day[df_day['aggressor_side'] == -1]['t'].to_numpy()

events = [buy_times, sell_times]

In [11]:
decay = 1/60 

In [12]:
from BivariateHawkes_optimized import BivariateHawkesOptimized

Numba not available. Install with: pip install numba


In [13]:
model = BivariateHawkesOptimized()

In [14]:
model.fit(buy_times, sell_times, optimize_beta=True)

In [15]:
results = model.get_results()

In [16]:
results

{'mu_buy': np.float64(1e-10),
 'mu_sell': np.float64(1.5922560349197657e-05),
 'alpha_bb': np.float64(11110.233936482926),
 'alpha_bs': np.float64(4328.096228617026),
 'alpha_sb': np.float64(3885.0443986418936),
 'alpha_ss': np.float64(9725.6813823859),
 'beta': np.float64(24563.963992568843),
 'br_bb': np.float64(0.4522980875498769),
 'br_bs': np.float64(0.17619697822087566),
 'br_sb': np.float64(0.15816031971945602),
 'br_ss': np.float64(0.3959328952496486),
 'spectral_radius': np.float64(0.5934129489553213),
 'total_branching': np.float64(1.1825882807398571),
 'log_likelihood': np.float64(1470278.0284006584),
 'is_stationary': np.True_}

In [20]:
pd.DataFrame({target_date: results}).T

,mu_buy,mu_sell,alpha_bb,alpha_bs,alpha_sb,alpha_ss,beta,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
2019-08-13,0.0,0.000016,11110.233936,4328.096229,3885.044399,9725.681382,24563.963993,0.452298,0.176197,0.15816,0.395933,0.593413,1.182588,1470278.028401,True


In [18]:
buy_times

array([2.42462160e-02, 1.18347168e-01, 3.17398071e-01, ...,
       2.42857539e+04, 2.42857539e+04, 2.42857539e+04])

In [21]:
all_day_results = {target_date: results}

In [22]:
all_day_results

{'2019-08-13': {'mu_buy': np.float64(1e-10),
  'mu_sell': np.float64(1.5922560349197657e-05),
  'alpha_bb': np.float64(11110.233936482926),
  'alpha_bs': np.float64(4328.096228617026),
  'alpha_sb': np.float64(3885.0443986418936),
  'alpha_ss': np.float64(9725.6813823859),
  'beta': np.float64(24563.963992568843),
  'br_bb': np.float64(0.4522980875498769),
  'br_bs': np.float64(0.17619697822087566),
  'br_sb': np.float64(0.15816031971945602),
  'br_ss': np.float64(0.3959328952496486),
  'spectral_radius': np.float64(0.5934129489553213),
  'total_branching': np.float64(1.1825882807398571),
  'log_likelihood': np.float64(1470278.0284006584),
  'is_stationary': np.True_}}

In [23]:
for target_date in dates[1:]:
    df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])
    market_open = pd.to_datetime((target_date) + ' 09:15:00')
    df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()
    buy_times = df_day[df_day['aggressor_side'] == +1]['t'].to_numpy()
    sell_times = df_day[df_day['aggressor_side'] == -1]['t'].to_numpy()
    events = [buy_times, sell_times]
    model = BivariateHawkesOptimized()
    model.fit(buy_times, sell_times, optimize_beta=True)
    r = model.get_results()
    all_day_results[target_date] = r
pd.DataFrame(all_day_results).T.to_csv('../data/all_day_results.csv')

OSError: Cannot save file into a non-existent directory: '..\data'

In [26]:
pd.DataFrame(all_day_results).T.to_csv('../data/all_day_results.csv')

In [27]:
pd.DataFrame(all_day_results).T.to_csv('all_day_results_{}.csv'.format(target_date))

In [29]:
from MarkedBivariateHawkes_optimized import MarkedBivariateHawkesOptimized

In [31]:
# Cell: Process all dates with marked Hawkes (similar to all_day_results)
all_day_marked_results = {}

for i, target_date in enumerate(dates):
    print(f"Processing {i+1}/{len(dates)}: {target_date}...")
    
    # Prepare data for the day
    df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])
    market_open = pd.to_datetime((target_date) + ' 09:15:00')
    df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()
    
    # Extract buy and sell events with volumes
    buy_mask = df_day['aggressor_side'] == +1
    sell_mask = df_day['aggressor_side'] == -1
    
    buy_times = df_day[buy_mask]['t'].to_numpy()
    sell_times = df_day[sell_mask]['t'].to_numpy()
    buy_volumes = df_day[buy_mask]['volume'].to_numpy()
    sell_volumes = df_day[sell_mask]['volume'].to_numpy()
    
    # Skip if insufficient data
    if len(buy_times) < 10 or len(sell_times) < 10:
        print(f"  Skipping: insufficient events")
        continue
    
    try:
        # Fit marked Hawkes model
        model = MarkedBivariateHawkesOptimized()
        model.fit(buy_times, sell_times, buy_volumes, sell_volumes, 
                  optimize_beta=True, verbose=False)
        
        # Store results
        all_day_marked_results[target_date] = model.get_results()
        pd.DataFrame(all_day_marked_results).T.to_csv('all_day_marked_results_{}.csv'.format(target_date))
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

print(f"\nCompleted: {len(all_day_marked_results)}/{len(dates)} dates")

# Cell: Save results as DataFrame
df_marked_results = pd.DataFrame(all_day_marked_results).T
df_marked_results.to_csv('../data/all_day_marked_results.csv')
print(f"Saved to: ../data/all_day_marked_results.c
sv")
print(df_marked_results.head())

Processing 1/10: 2019-08-13...
Processing 2/10: 2019-08-14...
Processing 3/10: 2019-08-16...
Processing 4/10: 2019-08-19...
Processing 5/10: 2019-08-20...
Processing 6/10: 2019-08-21...
Processing 7/10: 2019-08-22...
Processing 8/10: 2019-08-23...
Processing 9/10: 2019-08-26...
Processing 10/10: 2019-08-27...

Completed: 10/10 dates
Saved to: ../data/all_day_marked_results.csv
              mu_buy   mu_sell      alpha_bb      alpha_bs     alpha_sb  \
2019-08-13       0.0   0.00005   1413.224351   1351.644465   484.358717   
2019-08-14  0.000058       0.0   2972.043103    603.450777   647.627549   
2019-08-16  0.001694       0.0   1399.760427    289.769062   643.928139   
2019-08-19       0.0  0.001814   2503.635706    751.308251  1012.084692   
2019-08-20  0.030899       0.0  25357.324177  10285.932356  4820.213991   

                alpha_ss          beta  avg_mark     br_bb     br_bs  \
2019-08-13   1569.618928  10376.059644  2.716042  0.369926  0.353807   
2019-08-14   3214.557979 

In [32]:
pd.DataFrame(all_day_results).T

,mu_buy,mu_sell,alpha_bb,alpha_bs,alpha_sb,alpha_ss,beta,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
2019-08-13,0.0,0.000016,11110.233936,4328.096229,3885.044399,9725.681382,24563.963993,0.452298,0.176197,0.15816,0.395933,0.593413,1.182588,1470278.028401,True
2019-08-14,0.000011,0.0,48099.381852,31626.902075,33895.498939,45455.45169,99999.813684,0.480995,0.31627,0.338956,0.454555,0.795458,1.590775,1550959.230572,True
2019-08-16,0.000068,0.0,4372.033916,356.632035,142.494795,2008.722547,6856.359803,0.637661,0.052015,0.020783,0.292972,0.640769,1.003431,994694.178846,True
2019-08-19,0.0,0.000078,28664.668389,23017.421771,33093.052836,23789.058743,88075.556987,0.325455,0.261337,0.375735,0.270098,0.612355,1.232626,1042637.65104,True
2019-08-20,0.0,0.0,25399.988192,25188.971964,369.792795,25445.02027,51008.804132,0.497953,0.493816,0.00725,0.498836,0.558229,1.497855,1287516.942039,True
2019-08-21,0.0,0.000016,48383.310305,6742.665147,6984.647903,49660.39007,100000.0,0.483833,0.067427,0.069846,0.496604,0.559141,1.11771,1552346.165708,True
2019-08-22,0.0,0.000095,44952.360015,31921.193859,27910.666324,45055.379387,83601.905083,0.537695,0.381824,0.333852,0.538928,0.895345,1.792299,1352542.297152,True
2019-08-23,0.000376,0.0,25019.183007,2787.515184,2405.62688,5889.527255,50521.655286,0.495217,0.055175,0.047616,0.116574,0.502033,0.714582,1272864.913124,True
2019-08-26,0.000079,0.0,39366.477002,11141.328246,28240.157235,39356.352995,80812.365558,0.487134,0.137867,0.349453,0.487009,0.706566,1.461463,1401643.833905,True
2019-08-27,0.0,0.000024,6510.265654,5275.861555,714.876389,6021.775821,13669.339304,0.476268,0.385963,0.052298,0.440532,0.601593,1.35506,2123413.085598,True


In [33]:
pd.DataFrame(all_day_results).T[['br_bb', 'br_bs', 'br_sb', 'br_ss', 'spectral_radius', 'total_branching', 'log_likelihood', 'is_stationary']]

,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
2019-08-13,0.452298,0.176197,0.15816,0.395933,0.593413,1.182588,1470278.028401,True
2019-08-14,0.480995,0.31627,0.338956,0.454555,0.795458,1.590775,1550959.230572,True
2019-08-16,0.637661,0.052015,0.020783,0.292972,0.640769,1.003431,994694.178846,True
2019-08-19,0.325455,0.261337,0.375735,0.270098,0.612355,1.232626,1042637.65104,True
2019-08-20,0.497953,0.493816,0.00725,0.498836,0.558229,1.497855,1287516.942039,True
2019-08-21,0.483833,0.067427,0.069846,0.496604,0.559141,1.11771,1552346.165708,True
2019-08-22,0.537695,0.381824,0.333852,0.538928,0.895345,1.792299,1352542.297152,True
2019-08-23,0.495217,0.055175,0.047616,0.116574,0.502033,0.714582,1272864.913124,True
2019-08-26,0.487134,0.137867,0.349453,0.487009,0.706566,1.461463,1401643.833905,True
2019-08-27,0.476268,0.385963,0.052298,0.440532,0.601593,1.35506,2123413.085598,True


In [34]:
df_marked_results

,mu_buy,mu_sell,alpha_bb,alpha_bs,alpha_sb,alpha_ss,beta,avg_mark,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
2019-08-13,0.0,0.00005,1413.224351,1351.644465,484.358717,1569.618928,10376.059644,2.716042,0.369926,0.353807,0.126786,0.410864,0.603178,1.261383,1351100.134825,True
2019-08-14,0.000058,0.0,2972.043103,603.450777,647.627549,3214.557979,10530.310853,2.655542,0.749492,0.152179,0.163319,0.81065,0.94066,1.87564,1324903.832932,True
2019-08-16,0.001694,0.0,1399.760427,289.769062,643.928139,1500.988531,4998.900858,2.600939,0.728298,0.150767,0.335037,0.780968,0.980921,1.99507,989368.7902,True
2019-08-19,0.0,0.001814,2503.635706,751.308251,1012.084692,2724.352546,9037.600882,2.480952,0.687284,0.206245,0.277832,0.747874,0.958866,1.919235,882652.692502,True
2019-08-20,0.030899,0.0,25357.324177,10285.932356,4820.213991,24472.820543,100000.0,2.903157,0.736163,0.298617,0.139938,0.710484,0.928147,1.885202,1398866.893976,True
2019-08-21,0.0,0.009954,4146.171295,1055.775858,1277.848426,4887.623857,15304.862688,2.497078,0.676472,0.172256,0.208488,0.797444,0.935885,1.854661,1356264.773498,True
2019-08-22,0.0,0.001064,2359.33165,469.465901,935.59028,2714.553271,7941.134274,2.422804,0.719821,0.143232,0.285444,0.828198,0.983345,1.976696,1128120.718765,True
2019-08-23,0.00613,0.0,1011.323316,131.222265,277.351332,1193.114915,3745.769801,2.571068,0.694165,0.09007,0.190372,0.818945,0.901604,1.793552,1058740.036855,True
2019-08-26,0.002368,0.0,8029.707875,2193.063925,1648.505278,9464.41772,30228.878863,2.356927,0.626071,0.170992,0.128533,0.737935,0.840453,1.663531,1304165.228145,True
2019-08-27,0.0,0.000305,1535.116308,196.761947,207.427762,2566.615916,7606.161419,2.772518,0.559564,0.071722,0.075609,0.935556,0.949464,1.642451,2028976.670924,True


In [36]:
df_marked_results[['br_bb', 'br_bs', 'br_sb', 'br_ss', 'spectral_radius', 'total_branching', 'log_likelihood', 'is_stationary']]

,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
2019-08-13,0.369926,0.353807,0.126786,0.410864,0.603178,1.261383,1351100.134825,True
2019-08-14,0.749492,0.152179,0.163319,0.81065,0.94066,1.87564,1324903.832932,True
2019-08-16,0.728298,0.150767,0.335037,0.780968,0.980921,1.99507,989368.7902,True
2019-08-19,0.687284,0.206245,0.277832,0.747874,0.958866,1.919235,882652.692502,True
2019-08-20,0.736163,0.298617,0.139938,0.710484,0.928147,1.885202,1398866.893976,True
2019-08-21,0.676472,0.172256,0.208488,0.797444,0.935885,1.854661,1356264.773498,True
2019-08-22,0.719821,0.143232,0.285444,0.828198,0.983345,1.976696,1128120.718765,True
2019-08-23,0.694165,0.09007,0.190372,0.818945,0.901604,1.793552,1058740.036855,True
2019-08-26,0.626071,0.170992,0.128533,0.737935,0.840453,1.663531,1304165.228145,True
2019-08-27,0.559564,0.071722,0.075609,0.935556,0.949464,1.642451,2028976.670924,True


In [38]:
import os

In [ ]:
import pandas as pd
import numpy as np
from BivariateHawkes_optimized import BivariateHawkesOptimized

# 1. Define Helper to identify Aggressor Type
def get_aggressor_type(row):
    # Returns True if Aggressor is Algo, False if Non-Algo
    # Algo Flags: 0, 2 | Non-Algo Flags: 1, 3
    if row['aggressor_side'] == 1:
        return row['buy_algo'] in [0, 2]
    elif row['aggressor_side'] == -1:
        return row['sell_algo'] in [0, 2]
    return False

segmentation_results = []
dates = sorted(df['date'].unique())

print(f"Starting Segmentation Analysis for {len(dates)} days...")

for date in dates:
    print(f"Processing {date}...")
    
    # Filter for the day & Sort
    # Using stable sort to handle same-microsecond trades correctly
    df_day = df[df['date'] == date].sort_values(['timestamp', 'trade_number'], kind='stable')
    
    # 2. Vectorized Segmentation Mask
    # Logic: If Buy-Aggressor, check buy_algo. If Sell-Aggressor, check sell_algo.
    is_buy_aggressor = df_day['aggressor_side'] == 1
    is_sell_aggressor = df_day['aggressor_side'] == -1
    
    # Algo masks (Flags 0 or 2)
    algo_flags = [0, 2]
    is_algo_buy = is_buy_aggressor & df_day['buy_algo'].isin(algo_flags)
    is_algo_sell = is_sell_aggressor & df_day['sell_algo'].isin(algo_flags)
    is_algo_aggressor = is_algo_buy | is_algo_sell
    
    # Split Dataframes
    df_algo = df_day[is_algo_aggressor]
    df_human = df_day[~is_algo_aggressor]
    
    # 3. Prepare Event Arrays (Times need to be relative to open for numerical stability)
    # Algo Streams
    algo_buy_times = df_algo[df_algo['aggressor_side'] == 1]['timestamp'].values
    algo_sell_times = df_algo[df_algo['aggressor_side'] == -1]['timestamp'].values
    # Human Streams
    human_buy_times = df_human[df_human['aggressor_side'] == 1]['timestamp'].values
    human_sell_times = df_human[df_human['aggressor_side'] == -1]['timestamp'].values
    
    # Normalize times (start at 0)
    t_start = df_day['timestamp'].min()
    algo_buy_times = algo_buy_times - t_start
    algo_sell_times = algo_sell_times - t_start
    human_buy_times = human_buy_times - t_start
    human_sell_times = human_sell_times - t_start
    
    # 4. Fit Models (Only if enough data points exist)
    day_res = {'date': date, 'count_algo': len(df_algo), 'count_human': len(df_human)}
    
    # --- Fit Algo Model ---
    if len(algo_buy_times) > 100 and len(algo_sell_times) > 100:
        try:
            model_algo = BivariateHawkesOptimized()
            # Algo bounds: Allow very high beta (fast)
            model_algo.fit(algo_buy_times, algo_sell_times, optimize_beta=True)
            res_algo = model_algo.get_results()
            
            day_res['algo_beta'] = res_algo['beta']
            day_res['algo_half_life_us'] = (np.log(2) / res_algo['beta']) * 1e6
            day_res['algo_spectral_radius'] = res_algo['spectral_radius']
            day_res['algo_br_diagonal'] = res_algo['br_bb'] + res_algo['br_ss'] # Herding
        except Exception as e:
            print(f"  Algo fit failed: {e}")
            
    # --- Fit Human Model ---
    if len(human_buy_times) > 100 and len(human_sell_times) > 100:
        try:
            model_human = BivariateHawkesOptimized()
            # Human bounds: Beta might be lower, but we use same optimizer to find out
            model_human.fit(human_buy_times, human_sell_times, optimize_beta=True)
            res_human = model_human.get_results()
            
            day_res['human_beta'] = res_human['beta']
            day_res['human_half_life_us'] = (np.log(2) / res_human['beta']) * 1e6
            day_res['human_spectral_radius'] = res_human['spectral_radius']
            day_res['human_br_diagonal'] = res_human['br_bb'] + res_human['br_ss'] # Herding
        except Exception as e:
            print(f"  Human fit failed: {e}")
            
    segmentation_results.append(day_res)

# 5. Create Comparison DataFrame
df_seg_results = pd.DataFrame(segmentation_results)

# Calculate Ratio (Machine / Man)
df_seg_results['speed_ratio'] = df_seg_results['algo_beta'] / df_seg_results['human_beta']

print("\n--- Segmentation Results ---")
print(df_seg_results[['date', 'count_algo', 'count_human', 
                      'algo_half_life_us', 'human_half_life_us', 
                      'algo_spectral_radius', 'human_spectral_radius']].head())

In [40]:
algo_df = pd.DataFrame()
human_df = pd.DataFrame()

for f in os.listdir('../data/algo-vs-nonalgo'):
    if 'algo' in f:
        algo_df = pd.concat([algo_df, pd.read_csv('../data/algo-vs-nonalgo/' + f)])
    elif 'human' in f:
        human_df = pd.concat([human_df, pd.read_csv('../data/algo-vs-nonalgo/' + f)])

In [49]:
algo_df.rename(columns={"Unnamed: 0": "date"}, inplace=True)
algo_df.set_index("date", inplace=True)
algo_df

,mu_buy,mu_sell,alpha_bb,alpha_bs,alpha_sb,alpha_ss,beta,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
date,,,,,,,,,,,,,,,
2019-08-13,1.000000e-10,1.037190e-03,1864.444766,251.825442,673.452421,1694.062262,4318.036304,0.431781,0.058319,0.155963,0.392322,0.509442,1.038385,8.188353e+05,True
2019-08-14,1.000000e-10,1.293475e-06,28979.303389,210.173193,226.950145,28976.321684,58094.785873,0.498828,0.003618,0.003907,0.498777,0.502562,1.005129,9.747677e+05,True
2019-08-16,8.558101e-04,1.000000e-10,5373.544047,1486.268442,2144.098795,5194.601425,12005.436898,0.447593,0.123800,0.178594,0.432687,0.589020,1.182674,6.910911e+05,True
2019-08-19,1.000000e-10,9.565340e-04,19861.982629,18698.519071,18779.288439,20563.323822,44430.287019,0.447037,0.420851,0.422669,0.462822,0.876762,1.753379,6.373506e+05,True
2019-08-20,1.000000e-10,9.840151e-02,27758.627539,19329.981323,21507.576886,31040.004994,62997.960467,0.440627,0.306835,0.341401,0.492714,0.791374,1.581578,8.021317e+05,True
2019-08-21,1.000000e-10,1.176339e-02,8927.421497,3987.190847,4026.000467,10785.715478,22228.903251,0.401613,0.179370,0.181116,0.485211,0.628436,1.247310,8.895245e+05,True
2019-08-22,5.390653e-05,1.000000e-10,25272.587700,114.485254,25142.382986,25285.072298,50636.711069,0.499096,0.002261,0.496525,0.499343,0.532725,1.497225,8.267375e+05,True
2019-08-23,1.000000e-10,1.842920e-01,50000.000000,24352.437001,30985.396114,50000.000000,100000.000000,0.500000,0.243524,0.309854,0.500000,0.774694,1.553378,8.849105e+05,True
2019-08-26,1.011721e-02,1.000000e-10,21791.425566,5169.138395,2463.632301,18432.728103,33819.104960,0.644353,0.152847,0.072847,0.545039,0.711316,1.415085,8.021493e+05,True


In [47]:
human_df.rename(columns={"Unnamed: 0": "date"}, inplace=True)

In [50]:
human_df.set_index("date", inplace=True)
human_df    

,mu_buy,mu_sell,alpha_bb,alpha_bs,alpha_sb,alpha_ss,beta,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
date,,,,,,,,,,,,,,,
2019-08-13,1.000000e-10,6.572363e-05,17169.798409,16880.308238,16841.619653,17062.805509,34369.396397,0.499566,0.491144,0.490018,0.496453,0.988593,1.977181,532219.520248,True
2019-08-14,5.944113e-05,1.000000e-10,16331.287116,5821.140954,4732.708245,13859.790106,32601.421763,0.500938,0.178555,0.145169,0.425128,0.628434,1.249790,472520.987598,True
2019-08-16,1.000000e-10,2.151539e-04,25506.010830,25086.971611,680.697894,25542.297905,50860.985808,0.501485,0.493246,0.013383,0.502198,0.583091,1.510312,418712.007891,True
2019-08-19,1.000000e-10,4.148919e-05,17418.204132,16987.224526,16973.017950,17328.707442,34933.424117,0.498611,0.486274,0.485868,0.496049,0.983403,1.966803,358673.443920,True
2019-08-20,2.034588e-05,1.000000e-10,34334.733198,12346.149727,28789.151167,34340.856560,57840.201953,0.593614,0.213453,0.497736,0.593720,0.919616,1.898522,550910.180933,True
2019-08-21,1.000000e-10,2.745550e-05,3337.060055,1330.395698,1870.320632,3625.121453,7872.205751,0.423904,0.168999,0.237585,0.460496,0.643413,1.290985,461517.932875,True
2019-08-22,1.000000e-10,6.236576e-05,6248.376667,3036.096738,1977.010384,5995.812695,14612.204620,0.427614,0.207778,0.135299,0.410329,0.586860,1.181019,411708.990939,True
2019-08-23,4.974984e-05,1.000000e-10,29709.110330,614.393685,29366.224793,29711.497738,59523.319427,0.499117,0.010322,0.493357,0.499157,0.570498,1.501953,502827.982361,True
2019-08-26,1.516089e-04,1.000000e-10,10977.984900,3768.766666,4949.741461,10985.297229,24034.420474,0.456761,0.156807,0.205944,0.457065,0.636617,1.276577,511116.014326,True


In [43]:
segmentation_df = pd.read_csv('../data/algo-vs-nonalgo/segmentation_results_full.csv')

In [51]:
segmentation_df.drop(['Unnamed: 0'], axis=1, inplace=True)
segmentation_df.set_index('date', inplace=True)
segmentation_df


,count_algo,count_human,algo_beta,algo_half_life_us,algo_spectral_radius,algo_br_diagonal,human_beta,human_half_life_us,human_spectral_radius,human_br_diagonal,speed_ratio
date,,,,,,,,,,,
2019-08-13,103539,57850,4318.036304,160.523704,0.509442,0.824103,34369.396397,20.167569,0.988593,0.996020,0.125636
2019-08-14,103708,50748,58094.785873,11.931315,0.502562,0.997605,32601.421763,21.261256,0.628434,0.926066,1.781971
2019-08-16,80433,44630,12005.436898,57.736106,0.589020,0.880280,50860.985808,13.628269,0.583091,1.003683,0.236044
2019-08-19,68209,39093,44430.287019,15.600781,0.876762,0.909859,34933.424117,19.841948,0.983403,0.994661,1.271856
2019-08-20,83063,57437,62997.960467,11.002692,0.791374,0.933342,57840.201953,11.983831,0.919616,1.187333,1.089173
2019-08-21,98704,55812,22228.903251,31.182248,0.628436,0.886825,7872.205751,88.049932,0.643413,0.884400,2.823720
2019-08-22,89120,47039,50636.711069,13.688630,0.532725,0.998439,14612.204620,47.436181,0.586860,0.837943,3.465371
2019-08-23,88442,53205,100000.000000,6.931472,0.774694,1.000000,59523.319427,11.644969,0.570498,0.998274,1.680014
2019-08-26,85167,56162,33819.104960,20.495728,0.711316,1.189391,24034.420474,28.839771,0.636617,0.913826,1.407111


In [54]:
for col in segmentation_df.columns:
    if 'human' in col:
        segmentation_df.rename(columns={col: col.replace('human', 'non-algo')}, inplace=True)

In [55]:
segmentation_df

,count_algo,count_non-algo,algo_beta,algo_half_life_us,algo_spectral_radius,algo_br_diagonal,non-algo_beta,non-algo_half_life_us,non-algo_spectral_radius,non-algo_br_diagonal,speed_ratio
date,,,,,,,,,,,
2019-08-13,103539,57850,4318.036304,160.523704,0.509442,0.824103,34369.396397,20.167569,0.988593,0.996020,0.125636
2019-08-14,103708,50748,58094.785873,11.931315,0.502562,0.997605,32601.421763,21.261256,0.628434,0.926066,1.781971
2019-08-16,80433,44630,12005.436898,57.736106,0.589020,0.880280,50860.985808,13.628269,0.583091,1.003683,0.236044
2019-08-19,68209,39093,44430.287019,15.600781,0.876762,0.909859,34933.424117,19.841948,0.983403,0.994661,1.271856
2019-08-20,83063,57437,62997.960467,11.002692,0.791374,0.933342,57840.201953,11.983831,0.919616,1.187333,1.089173
2019-08-21,98704,55812,22228.903251,31.182248,0.628436,0.886825,7872.205751,88.049932,0.643413,0.884400,2.823720
2019-08-22,89120,47039,50636.711069,13.688630,0.532725,0.998439,14612.204620,47.436181,0.586860,0.837943,3.465371
2019-08-23,88442,53205,100000.000000,6.931472,0.774694,1.000000,59523.319427,11.644969,0.570498,0.998274,1.680014
2019-08-26,85167,56162,33819.104960,20.495728,0.711316,1.189391,24034.420474,28.839771,0.636617,0.913826,1.407111


In [58]:
segmentation_df.algo_beta.mean(), segmentation_df['non-algo_beta'].mean()

(np.float64(43957.10362511883), np.float64(32112.30044347296))

In [52]:
algo_df.to_csv("../data/algo_df.csv", index=False)
human_df.to_csv("../data/human_df.csv", index=False)
segmentation_df.to_csv("../data/segmentation_df.csv", index=False)